In [2]:
!pip install -U weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.6 MB/s eta 0:00:00


# Connect HuggineFace


In [3]:
import weaviate
from weaviate.classes.config import Configure, DataType, Property

# 1. Provide your token here
hf_token = "your_huggingface_token_here"

client = weaviate.connect_to_embedded(
    version="1.32.2",
    headers={"X-HuggingFace-Api-Key": hf_token} # Pass token to the client
)

try:
    client.collections.create(
        name="Article",
        properties=[Property(name="content", data_type=DataType.TEXT)],
        # Use a specific model to ensure it works
        vector_config=Configure.Vectors.text2vec_huggingface(
            model="sentence-transformers/all-MiniLM-L6-v2"
        )
    )

    # ... rest of your insert code ...
    articles = client.collections.get("Article")
    articles.data.insert_many([
        {"content": "Vector databases enable semantic search"},
        {"content": "Machine learning models generate embeddings"},
    ])
    print("Success!")
finally:
    client.close()

INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.32.2/weaviate-v1.32.2-Linux-amd64.tar.gz


KeyboardInterrupt: 

In [4]:
!pip install -U weaviate-client transformers pillow requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfully uninstalled pillow-11.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
gradio 5.50.0 requires pillow<12.0,>=8.0, but you have pillow 12.1.0 which is incompatible.


In [15]:
!pip uninstall Pillow -y
!pip install Pillow==9.5.0

Found existing installation: pillow 12.1.0
Uninstalling pillow-12.1.0:
  Would remove:
    /usr/local/lib/python3.12/dist-packages/PIL/*
    /usr/local/lib/python3.12/dist-packages/pillow-12.1.0.dist-info/*
    /usr/local/lib/python3.12/dist-packages/pillow.libs/libXau-154567c4.so.6.0.0
    /usr/local/lib/python3.12/dist-packages/pillow.libs/libavif-01e67780.so.16.3.0
    /usr/local/lib/python3.12/dist-packages/pillow.libs/libbrotlicommon-c55a5f7a.so.1.2.0
    /usr/local/lib/python3.12/dist-packages/pillow.libs/libbrotlidec-b57ddf63.so.1.2.0
    /usr/local/lib/python3.12/dist-packages/pillow.libs/libfreetype-ee1c40c4.so.6.20.4
    /usr/local/lib/python3.12/dist-packages/pillow.libs/libharfbuzz-0692f733.so.0.61230.0
    /usr/local/lib/python3.12/dist-packages/pillow.libs/libjpeg-18a3f47f.so.62.4.0
    /usr/local/lib/python3.12/dist-packages/pillow.libs/liblcms2-cc10e42f.so.2.0.17
    /usr/local/lib/python3.12/dist-packages/pillow.libs/liblzma-61b1002e.so.5.8.2
    /usr/local/lib/python3

In [16]:
!fuser -k 8079/tcp
!fuser -k 50050/tcp

In [17]:
import weaviate
from weaviate.classes.config import Configure, DataType, Property
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
import torch
import io
from IPython.display import display, Image as IPImage

# 1. Load CLIP Model (Standard Multimodal Model)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_embeddings(text=None, image_url=None):
    # CRITICAL: We need a browser-like header to prevent Unsplash from blocking us
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    if image_url:
        try:
            # Use a timeout to prevent hanging
            response = requests.get(image_url, stream=True, headers=headers, timeout=10)
            response.raise_for_status() # Raise error for 403, 404, etc.

            # Load into BytesIO and verify it's actually an image
            img_bytes = io.BytesIO(response.content)
            img = Image.open(img_bytes).convert("RGB")

            inputs = processor(images=img, return_tensors="pt").to(device)
            return model.get_image_features(**inputs).detach().cpu().numpy()[0].tolist()
        except Exception as e:
            print(f"Skipping {image_url} due to error: {e}")
            return None

    if text:
        inputs = processor(text=[text], return_tensors="pt").to(device)
        return model.get_text_features(**inputs).detach().cpu().numpy()[0].tolist()

# 2. Start Weaviate
client = weaviate.connect_to_embedded(version="1.32.2")

try:
    if client.collections.exists("MultiModal"):
        client.collections.delete("MultiModal")

    collection = client.collections.create(
        name="MultiModal",
        properties=[
            Property(name="label", data_type=DataType.TEXT),
            Property(name="url", data_type=DataType.TEXT),
        ],
        vectorizer_config=Configure.Vectorizer.none()
    )

    # High-quality Unsplash URLs
    image_data = [
        {"label": "Forest Cabin", "url": "https://images.unsplash.com/photo-1449156003053-c30650af80f3?w=600"},
        {"label": "Neon City", "url": "https://images.unsplash.com/photo-1477959858617-67f85cf4f1df?w=600"},
        {"label": "Golden Retriever", "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=600"}
    ]

    # 3. Safe Batch Import
    with collection.batch.dynamic() as batch:
        for item in image_data:
            vector = get_embeddings(image_url=item['url'])
            if vector: # Only add if embedding succeeded
                print(f"Success: Indexed {item['label']}")
                batch.add_object(properties=item, vector=vector)

    # 4. Multimodal Search
    search_query = "futuristic metropolis with lights"
    print(f"\n--- Searching for: '{search_query}' ---")

    query_vector = get_embeddings(text=search_query)
    results = collection.query.near_vector(near_vector=query_vector, limit=1)

    if results.objects:
        top_res = results.objects[0].properties
        print(f"Top Match: {top_res['label']}")
        display(IPImage(url=top_res['url']))

finally:
   # client.close()
   pass

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.32.2/weaviate-v1.32.2-Linux-amd64.tar.gz
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 3408


Skipping https://images.unsplash.com/photo-1449156003053-c30650af80f3?w=600 due to error: 404 Client Error: Not Found for url: https://images.unsplash.com/photo-1449156003053-c30650af80f3?w=600
Skipping https://images.unsplash.com/photo-1477959858617-67f85cf4f1df?w=600 due to error: property 'mode' of 'JpegImageFile' object has no setter
Skipping https://images.unsplash.com/photo-1552053831-71594a27632d?w=600 due to error: property 'mode' of 'JpegImageFile' object has no setter

--- Searching for: 'futuristic metropolis with lights' ---


In [20]:
 from transformers import CLIPProcessor, CLIPModel

## **Detailed Architectural Breakdown**

### **I. The CLIP Embedding Model**

Traditional image search relies on "tags" (e.g., a human labeling a photo as "dog"). CLIP (Contrastive Language-Image Pre-training) works differently. It was trained on 400 million image-text pairs from the internet.

* **Shared Embedding Space:** It learns that the visual representation of a "mountain" and the written word "mountain" should exist at the same coordinate in a high-dimensional space (512 dimensions in this case).  
* **Zero-Shot Ability:** Because it understands concepts, you can search for things it was never explicitly labeled for, like "a lonely atmosphere" or "vibrant colors."

### **II. Vector Database (Weaviate)**

A standard database (like SQL) is good at finding exact matches (e.g., WHERE id=10). However, it cannot "calculate" how similar two images are.

* **Vectors as Coordinates:** Weaviate treats every image as a point in a 512-dimensional room.  
* **HNSW Indexing:** To search 200 images is easy, but to search 200 million, Weaviate uses an algorithm called **HNSW (Hierarchical Navigable Small World)**. It creates a "map" of the points so it can find the closest match without checking every single entry.  
* **NearVector:** When we perform near\_vector, Weaviate calculates the **Cosine Distance** between your text query and the stored images. The smaller the distance, the more relevant the image.

### **III. The Workflow Summary**

1. **Ingestion:** Image → CLIP Vision Encoder → Vector → Weaviate.  
2. **Query:** Text → CLIP Text Encoder → Vector.  
3. **Retrieval:** Query Vector → Weaviate Search → Top $K$ Matches.



In [19]:
!fuser -k 8079/tcp
!fuser -k 50050/tcp

8079/tcp:             3408


In [21]:
import weaviate
from weaviate.classes.config import Configure, DataType, Property
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
import torch
import io
from IPython.display import display, Image as IPImage

# 1. Setup CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_embeddings(image_url):
    headers = {"User-Agent": "Mozilla/5.0"}
    try:
        response = requests.get(image_url, stream=True, headers=headers, timeout=5)
        img = Image.open(io.BytesIO(response.content)).convert("RGB")
        inputs = processor(images=img, return_tensors="pt").to(device)
        return model.get_image_features(**inputs).detach().cpu().numpy()[0].tolist()
    except Exception as e:
        return None

# 2. Connect and Reset Collection
client = weaviate.connect_to_embedded(version="1.32.2")

try:
    if client.collections.exists("Gallery"):
        client.collections.delete("Gallery")

    gallery = client.collections.create(
        name="Gallery",
        vectorizer_config=Configure.Vectorizer.none(),
        properties=[
            Property(name="image_id", data_type=DataType.INT),
            Property(name="url", data_type=DataType.TEXT)
        ]
    )

    # 3. Batch Index 200 Images
    print("🚀 Starting indexing of 200 images...")
    with gallery.batch.dynamic() as batch:
        for i in range(200):
            # Each ID gives a unique image from the Picsum/Unsplash collection
            url = f"https://picsum.photos/id/{i+10}/600/400"
            vector = get_embeddings(url)

            if vector:
                batch.add_object(
                    properties={"image_id": i, "url": url},
                    vector=vector
                )
                if (i + 1) % 10 == 0:
                    print(f"✅ Indexed {i + 1}/200 images")

    # 4. Multimodal Search
    query_text = "a photograph of a mountain or landscape"
    print(f"\n🔍 Searching for: '{query_text}'")

    # Encode the text query
    text_inputs = processor(text=[query_text], return_tensors="pt").to(device)
    query_vector = model.get_text_features(**text_inputs).detach().cpu().numpy()[0].tolist()

    results = gallery.query.near_vector(near_vector=query_vector, limit=3)

    for obj in results.objects:
        print(f"Match found (ID: {obj.properties['image_id']})")
        display(IPImage(url=obj.properties['url'], width=300))

finally:
    #client.close()
    pass

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 3502


🚀 Starting indexing of 200 images...

🔍 Searching for: 'a photograph of a mountain or landscape'


In [ ]:
query_text = "a cat"
print(f"\n🔍 Searching for: '{query_text}'")

# Encode the text query
text_inputs = processor(text=[query_text], return_tensors="pt").to(device)
query_vector = model.get_text_features(**text_inputs).detach().cpu().numpy()[0].tolist()

results = gallery.query.near_vector(near_vector=query_vector, limit=3)

for obj in results.objects:
    print(f"Match found (ID: {obj.properties['image_id']})")
    display(IPImage(url=obj.properties['url'], width=300))




🔍 Searching for: 'a cat'
Match found (ID: 30)


Match found (ID: 84)


Match found (ID: 104)
